In [1]:
import numpy as np
import pandas as pd

In [2]:
!ls

data  Untitled.ipynb


In [3]:
!ls data

sample_adv.txt	test.csv  train.csv


In [4]:
DATA_DIR = 'data/'
FULL_LINES = 29989752
TRAIN_LINES = int(FULL_LINES * 0.5)
VALID_LINES = FULL_LINES - TRAIN_LINES
print('FULL_LINES:', FULL_LINES)
print('TRAIN_LINES:', TRAIN_LINES)
print('VALID_LINES:', VALID_LINES)

FULL_LINES: 29989752
TRAIN_LINES: 14994876
VALID_LINES: 14994876


In [5]:
with open(DATA_DIR + 'train.csv') as f:
    fea_names = f.readline()[:-1].split(';')
    line = f.readline()[:-1]
    print(line)
    features = line.split(';')
    for i in range(len(fea_names)):
        print(f'{fea_names[i]} --- {i} --- {features[i]}')

1379278800;0;2733540231;3500392421;4454;15573;11;995;2;176;15;671;384,382,96,88,185,49,385,268,448,438,279,420,124,123,5,3,17,133,71,409,330,59,57,302,304,395,275,151,113,99,154,155,76,412,139,333,332,335,334,399;;;32;0;0;106
timestamp --- 0 --- 1379278800
label --- 1 --- 0
C1 --- 2 --- 2733540231
C2 --- 3 --- 3500392421
C3 --- 4 --- 4454
C4 --- 5 --- 15573
C5 --- 6 --- 11
C6 --- 7 --- 995
C7 --- 8 --- 2
C8 --- 9 --- 176
C9 --- 10 --- 15
C10 --- 11 --- 671
CG1 --- 12 --- 384,382,96,88,185,49,385,268,448,438,279,420,124,123,5,3,17,133,71,409,330,59,57,302,304,395,275,151,113,99,154,155,76,412,139,333,332,335,334,399
CG2 --- 13 --- 
CG3 --- 14 --- 
l1 --- 15 --- 32
l2 --- 16 --- 0
C11 --- 17 --- 0
C12 --- 18 --- 106


In [6]:
from tqdm import tqdm

In [7]:
labels = []

In [8]:
def make_vm_from_csv(csv, vw, labels_=False, test=False):
    global labels
    if labels_:
        labels = []
    with open(DATA_DIR + csv) as f, open(vw, 'w') as out_f:
        f.readline()
        for line in tqdm(f, total=FULL_LINES):
            features = line[:-1].split(';')
            if test:
                label=-1
            else:
                label = int(features[1]) * 2 - 1
            if labels_:
                labels.append(int(features[1]))
            res = f'{label} '

            res += '|i '
            res += f'l1:{features[15]} l2:{features[16]} '
    #         res += '|j '
    #         res += f'l1_{features[15]} l2_{features[16]} '

            res += '|c '
            for i in range(2, 12):
                res += f'C{i-1}_{features[i]} '
            res += f'C11_{features[17]} '
            res += f'C12_{features[18]} '

            res += '|l '
            if features[12] != '':
                for idx, fea in enumerate(features[12].split(',')):
                    res += f'CG1_{0}_{fea} '

            res += '|m '
            if features[13] != '':
                for idx, fea in enumerate(features[13].split(',')):
                    res += f'CG2_{0}_{fea} '

            res += '|n '
            if features[14] != '':
                for idx, fea in enumerate(features[14].split(',')):
                    res += f'CG3_{0}_{fea} '

            out_f.write(res + '\n')

In [9]:
make_vm_from_csv('train.csv', 'full.vw', True)

100%|██████████| 29989752/29989752 [08:50<00:00, 56535.95it/s]


In [10]:
# with open('full.vw', 'r') as full, open('train.vw', 'w') as train, open('valid.vw', 'w') as valid:
#     c = 0
#     for line in tqdm(full, total=FULL_LINES):
#         c += 1
#         if c < TRAIN_LINES:
#             train.write(line)
#         else:
#             valid.write(line)

In [11]:
# !vw train.vw -f valid_model.vw --loss_function=logistic --link=logistic --passes 1 -c --cubic ccc -q ll -q mm -q nn --adaptive --normalized --l1 0.00000001 --l2 0.0000001 -b 25 

In [12]:
# !vw -d valid.vw -t -i valid_model.vw --loss_function=logistic --link=logistic -p valid_pred.txt

In [13]:
# print(len(labels))
# valid_labels = labels[TRAIN_LINES - 1:]
# valid_pred = []
# with open('valid_pred.txt', 'r') as f:
#     count = 0
#     for line in f:
#         valid_pred.append(float(line[:-1]))
#         count += 1
#     print(len(valid_labels))
#     print(len(valid_pred))
# valid_labels_np = np.array(valid_labels)
# valid_pred_np = np.array(valid_pred)
# print(valid_labels_np.shape)
# print(valid_pred_np.shape)

In [14]:
# from sklearn.metrics import log_loss
# print(log_loss(valid_labels_np, valid_pred_np))

0.016967645176882876

In [17]:
!vw full.vw -f model.vw --loss_function=logistic --link=logistic --passes 2 -c --cubic ccc -q ll -q mm -q nn --adaptive --normalized --l1 0.00000001 --l2 0.0000001 -b 20 

creating quadratic features for pairs: ll mm nn 
creating cubic features for triples: ccc 
using l1 regularization = 1e-08
using l2 regularization = 1e-07
final_regressor = model.vw
Num weight bits = 20
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = full.vw.cache
Reading datafile = full.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.5000     1238
0.354293 0.015439            2            2.0  -1.0000   0.0153      873
0.187282 0.020270            4            4.0  -1.0000   0.0017     1513
0.099674 0.012067            8            8.0  -1.0000   0.0000     3546
0.050758 0.001842           16           16.0  -1.0000   0.0000      677
0.026572 0.002386           32           32.0  -1.0000   0.0000      980
0.020433 0.014294           64           64.0  -1.0000   0.000

  6%|▌         | 1795632/29989752 [01:00<14:52, 31603.97it/s]

0.044859 0.047025       262144       262144.0  -1.0000   0.0000     1602
0.041378 0.037896       524288       524288.0  -1.0000   0.0001      814
0.037000 0.032622      1048576      1048576.0  -1.0000   0.0383     1198
0.030592 0.024185      2097152      2097152.0  -1.0000   0.0004      783
0.024073 0.017554      4194304      4194304.0  -1.0000   0.0029     1080
0.021247 0.018420      8388608      8388608.0  -1.0000   0.0001     1280
0.018458 0.015670     16777216     16777216.0  -1.0000   0.0004     1009
0.016759 0.016759     33554432     33554432.0  -1.0000   0.0003     2267 h

finished run
number of examples per pass = 26990777
passes used = 2
weighted example sum = 53981554.000000
weighted label sum = -53706702.000000
average loss = 0.015166 h
best constant = -5.970778
best constant's loss = 0.017749
total feature number = 73598626406


In [18]:
make_vm_from_csv('test.csv', 'test.vw', False, True)


  2%|▏         | 725510/29989752 [00:12<08:57, 54486.88it/s]


  5%|▍         | 1446852/29989752 [00:25<08:31, 55835.55it/s]


  7%|▋         | 2170121/29989752 [00:38<08:08, 56947.15it/s]


 10%|▉         | 2893643/29989752 [00:51<07:58, 56571.16it/s]


 12%|█▏        | 3618534/29989752 [01:03<07:52, 55757.14it/s]


...

 60%|█████▉    | 17881697/29989752 [05:17<03:44, 53895.81it/s]


 62%|██████▏   | 18578729/29989752 [05:29<03:27, 54977.10it/s]


 64%|██████▍   | 19289942/29989752 [05:42<03:10, 56191.72it/s]


 67%|██████▋   | 19988008/29989752 [05:55<02:57, 56411.55it/s]


 68%|██████▊   | 20317220/29989752 [06:01<02:51, 56243.10it/s]


In [19]:
!vw -d test.vw -t -i model.vw --loss_function=logistic --link=logistic -p pred.txt

creating quadratic features for pairs: ll mm nn 
creating cubic features for triples: ccc 
only testing
predictions = pred.txt
ignoring duplicate option: '--link=logistic'
Num weight bits = 20
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.002456 0.002456            1            1.0  -1.0000   0.0025     1117
0.001449 0.000442            2            2.0  -1.0000   0.0004     2201
0.004047 0.006645            4            4.0  -1.0000   0.0124      732
0.005147 0.006248            8            8.0  -1.0000   0.0005      755
0.003345 0.001543           16           16.0  -1.0000   0.0003     2201
0.002921 0.002496           32           32.0  -1.0000   0.0065      729
0.003204 0.003487           64           64.0  -1.0000   0.0008     2030
0.003494 0.003785          128          128.

In [20]:
with open('pred.txt', 'r') as in_f, open('sub0.txt', 'w') as out_f:
    out_f.write("Id,Click\n")
    for idx, line in enumerate(tqdm(in_f)):
        out_f.write("{},{}".format(idx+1, line))


0it [00:00, ?it/s]
117955it [00:00, 1179542.69it/s]
233021it [00:00, 1170725.41it/s]
364388it [00:00, 1210231.69it/s]
493969it [00:00, 1234692.13it/s]
632365it [00:00, 1275977.22it/s]
772080it [00:00, 1310062.65it/s]
915345it [00:00, 1344575.70it/s]
1059940it [00:00, 1373462.84it/s]
1205708it [00:00, 1397687.61it/s]
1350889it [00:01, 1413494.25it/s]
1495479it [00:01, 1423059.82it/s]
1639812it [00:01, 1429079.22it/s]
1781312it [00:01, 1409209.58it/s]
1922607it [00:01, 1410327.00it/s]
2069612it [00:01, 1427727.49it/s]
2215367it [00:01, 1436542.68it/s]
2361491it [00:01, 1443862.14it/s]
2509180it [00:01, 1453610.66it/s]
2656271it [00:01, 1458754.89it/s]
2803585it [00:02, 1463038.14it/s]
2949842it [00:02, 1462757.85it/s]
3098297it [00:02, 1469226.36it/s]
3245906it [00:02, 1471276.01it/s]
3393269it [00:02, 1471979.74it/s]
3540460it [00:02, 1452937.72it/s]
3685805it [00:02, 1431390.31it/s]
3829689it [00:02, 1433616.49it/s]
3975221it [00:02, 1440058.66it/s]
4122091it [00:02, 1448532.46it/s]
4